In [ ]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
    !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
    !git fetch --all
    !git reset --hard origin/master

In [ ]:
import os
cwd = os.getcwd()
if cwd[0:2] != "C:":
    from google.colab import drive
    drive.mount('/content/gdrive',force_remount=True)
    %cd "/content/gdrive/My Drive/Projects"
#     !git clone https://github.com/ai357060/modelF.git
    import sys
    root_dir="/content/gdrive/My Drive/Projects/modelF"
    sys.path.append(root_dir)
    %cd "/content/gdrive/My Drive/Projects/modelF/wm"
#     !git pull

In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import importlib
import tradestats
import pandas as pd
from datetime import datetime

In [ ]:
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz

In [ ]:
importlib.reload(tradestats)
from tradestats import *

In [ ]:
prices = loaddata_1D('AUDUSD_Candlestick_1_D_BID_03.08.2003-24.10.2021.csv')

In [ ]:
importlib.reload(tradestats)
from tradestats import *
paKey = [1]
dictList = []
colFeat = []
keyList = []
paDict   = priceaction(prices, paKey)
dictList.append(paDict.df); colFeat.append('pa'); keyList.append(paKey)

C:\Users\pioo\notebooks\wm\tradestats.py:271: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['open_prev']  = prices['open'].shift(1)
C:\Users\pioo\notebooks\wm\tradestats.py:272: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['high_prev']  = prices['high'].shift(1)
C:\Users\pioo\notebooks\wm\tradestats.py:273: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable

In [ ]:
# prices.dropna(inplace=True)
masterFrame = pd.DataFrame(index=prices.index)
masterFrame[['id','year','month','day','date','weekday','open','high','low','close','volume']] = prices[['id','year','month','day','date','weekday','open','high','low','close','volume']]

In [ ]:
for i in range(0,len(dictList)):#iteracja po funkcjach
    for j in keyList[i]: #iteracja po okresach
        for k in list(dictList[i][j]): #iteracja po kolumnach danej funkcji w okresie
            colID = colFeat[i] + str(j) + k
            masterFrame[colID] = dictList[i][j][k]

In [ ]:
# masterFrame.to_csv(sep=';',path_or_buf='../Data/masterFrame.csv',date_format="%Y-%m-%d",index = False,na_rep='')

In [ ]:
masterFrame = masterFrame.dropna()

In [ ]:
masterFrame.tail(5)

,id,year,month,day,date,weekday,open,high,low,close,volume,pa1pa,pa1body,pa1upwick,pa1downwick
4748,4748,2021,10,18,2021-10-18 21:00:00,0,0.74146,0.74370,0.73781,0.74102,96688.34,0,0.00044,0.00224,0.00321
4749,4749,2021,10,19,2021-10-19 21:00:00,1,0.74080,0.74851,0.74074,0.74715,93228.11,3,0.00635,0.00136,0.00006
4750,4750,2021,10,20,2021-10-20 21:00:00,2,0.74715,0.75222,0.74647,0.75144,94912.55,0,0.00429,0.00078,0.00068
4751,4751,2021,10,21,2021-10-21 21:00:00,3,0.75128,0.75460,0.74581,0.74655,97404.62,-2,0.00473,0.00332,0.00074
4752,4752,2021,10,22,2021-10-22 21:00:00,4,0.74654,0.75118,0.74533,0.74631,123259.25,0,0.00023,0.00464,0.00098


In [ ]:
importlib.reload(tradestats)
from tradestats import *
starttime = datetime.now()
# alltrades = preparetrades_brut_tsl(masterFrame,[1],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],2016,2017)
alltrades = preparetrades_brut_tp(masterFrame,[1],[0.001,0.002,0.003,0.004,0.005,0.006],[0.001,0.002,0.003,0.004,0.005,0.006],2016,2017)
endtime = datetime.now()
print(str(endtime - starttime))

-1
-2
-2
-3
-4
-6
-2
-2
-3
-4
-5
-6
-4
-5
-5
-5
-6
-8
-4
-5
-5
-5
-6
-8
-4
-5
-5
-8
-9
-11
-6
-6
-9
-9
-10
-14
0:00:19.117986


In [ ]:
importlib.reload(tradestats)
from tradestats import *
trades = cleartrades_brut(alltrades.copy(),True)
# trades,alltrades = preparetrades(masterFrame,[-1,1],[5],[13],[0.002],True)

In [ ]:
#____________________________________________________________________________________________________________________
#
#
alltrades.head(2)

,id,year,month,day,date,weekday,open,high,low,close,...,tp,stoploss,closeindex,tpcloseindex,closeprice,tpcloseprice,slindex,slprice,profit,profit1
3242,3242,2016,1,4,2016-01-04 22:00:00,0,0.72836,0.73000,0.71556,0.71901,...,10.0,0.72736,3242.0,-1.0,0.72736,-1.0,3242.0,0.71556,-10.0,-1
3243,3243,2016,1,5,2016-01-05 22:00:00,1,0.71896,0.72143,0.71319,0.71591,...,10.0,0.71796,3243.0,-1.0,0.71796,-1.0,3243.0,0.71319,-10.0,-1


In [ ]:
importlib.reload(tradestats)
from tradestats import *
stats1 = runstats_brut_v3(alltrades)

brut_2015_2021_3_
allexecs:  36
finish:         2021-11-01 20:13:22.676580
duration:       0:00:01.335427
